# Спрос на рынке жилья (за счет собственных средств)

In [1]:
from IPython.display import Markdown as md
import pandas as pd
import numpy as np

import patsy
import statsmodels.api as sm
import statsmodels.formula.api as smf

from scipy.integrate import quad
from scipy.stats import rv_continuous, norm

%run ../COMMON/common.ipynb # загрузка общих функций и констант, все, что оттуда, должно иметь префикс common.

conWork = sa.create_engine('sqlite+pysqlite:///{db_name}'.format(db_name=common.strYearDBPath)) # connection к рабочей базе данных
conWorkEx = sa.create_engine('sqlite+pysqlite:///{db_name}'.format(db_name=common.strExYearDBPath)) # connection к к рабочей базе данных экзогенных переменных
conWorkExH=sa.create_engine('sqlite+pysqlite:///{db_name}'.format(db_name=common.strExParamDBPath)) # connection к рабочей базе данных экзогенных параметров
conWorkSvod=sa.create_engine('sqlite+pysqlite:///{db_name}'.format(db_name=common.strSvodDBPath)) # connection к рабочей базе данных СВОД


## Установка временных точек

In [2]:
iFirstFactYear=2008
iLastFactYear=2019

iFirstForecastYear=iLastFactYear+1
iFirstForecastYear=2030

## Загрузка исходных данных

Исходные данные:
  - Фактические значения - из базы данных `year.sqlite3`;
  - Экзогенные - из базы данных `exog_year.sqlite3`
  - Задаваемые вручную - из базы данных `exog_param.sqlite3`
  - Результаты других моделей - из базы данных `svod.sqlite3`. *В эту же базу записываем результаты данного блокнота*. А даном блокноте используются результаты блока **"Цены и себестоимость"**

In [23]:
lstYearCodes=['squareddy_ss_x', 'CPIAv', 'Pop_x', 'HHAv', 'Unmpl_s', 'DispInc_rI', 'Inc_x', 'p_ProbDef', 
              'UZUss_x', 'MEPop', 'DispPop', 'LivMin', 'p_superrich', 'sdelkiddy_ss_x', 'sdelkikp_ss_x'] # список экзогенных и фактических данных
 

lstSvod=['price1mddy_alt_x', 'price1mall_alt_x'] # результаты из блока "Цены и себестоимость"

pdfAct=(pd.read_sql(common.make_SELECT_YEAR_string(lstYearCodes), con=conWork)
        .pipe(common.make_frame)
        .pipe(common.scale, list_fields=['Inc_x',], multiplier=1e3)
        .pipe(common.scale, list_fields=['Unmpl_s',], multiplier=100))
pdfExog=pd.read_sql(common.make_SELECT_YEAR_string(lstYearCodes), con=conWorkEx).pipe(common.make_frame)
PdfExogHandle=pd.read_sql(common.make_SELECT_YEAR_string(lstYearCodes), con=conWorkExH).pipe(common.make_frame)

pdfSvod=(pd.read_sql(common.make_SELECT_YEAR_string(lstSvod), con=conWorkSvod)
         .pipe(common.make_frame)
        .pipe(common.scale, list_fields=lstSvod, multiplier=1e3))

pdfWork=pd.concat([pdfAct, pdfExog[~pdfExog.index.isin(pdfAct.index)]]).combine_first(PdfExogHandle).join(pdfSvod)

pdfWork

,CPIAv,DispPop,HHAv,Inc_x,LivMin,MEPop,Pop_x,UZUss_x,Unmpl_s,p_ProbDef,p_superrich,sdelkiddy_ss_x,sdelkikp_ss_x,squareddy_ss_x,price1mall_alt_x,price1mddy_alt_x
date,,,,,,,,,,,,,,,,
2004,1.108812,NaN,2.677054,6410.300000,2376.0,NaN,144.067300,NaN,7.714864,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2005,1.127055,0.756100,2.661470,8088.300000,3018.0,8.711611,143.518814,NaN,7.153243,NaN,0.02,143935.804120,1.334440,NaN,21048.567265,18491.692819
2006,1.096830,0.769907,2.645886,10154.800000,3422.0,8.928732,143.049637,NaN,7.062219,NaN,0.02,152925.400209,1.417784,NaN,28597.165335,24937.896936
2007,1.089881,0.785663,2.630302,12540.200000,3847.0,9.129498,142.805114,NaN,6.029393,0.10,0.02,162476.446858,1.506332,NaN,37938.493305,32765.902784
2008,1.141048,0.782878,2.614717,14863.600000,4593.0,9.301331,142.742366,9.763312,6.241192,0.10,0.02,507312.838863,1.265722,NaN,44576.706072,37788.915681
2009,1.116977,0.783533,2.599133,16895.000000,5153.0,9.427587,142.785348,10.421675,8.252236,0.10,0.02,282607.252907,1.182876,NaN,42565.501231,36473.861415
2010,1.068508,0.782280,2.583549,18958.400000,5688.0,9.546797,142.849468,11.124432,7.344543,0.10,0.02,117507.304908,1.655528,7.060626,41870.584649,35853.805771
2011,1.084634,0.775441,2.544156,20780.000000,6369.0,9.644999,142.960908,11.874579,6.495643,0.10,0.02,161405.151036,1.983726,9.382939,44815.617500,37897.472700
2012,1.050667,0.781314,2.504763,23221.100000,6510.0,9.747765,143.201721,12.675309,5.458664,0.10,0.02,204776.640332,2.074330,11.554604,50629.950501,41865.338092


# Модель 1 - Площадь строящегося жилья, приобретенного за счет собственных средств

**Регрессоры:**
  - Потенциальный спрос на жилье (без лага с весом 0.5, с лагом 1 год с весом 0.5) - \_PDHouses\_w (**расчетный**)
  - Уровень безработицы - Ln(Unmpl\_s) (**из базы данных**)
  - Дамми-переменная на 2014 и 2019 гг. 
     <br><br>   
<center>  $\large squareddy\_ss\_x \sim \_PDHouses\_w + Ln(Unmpl\_s) + D_{(2014, 2019)} - 1 $  </center>

## 1. Потенциальный спрос на жилье (без лага с весом 0.5, с лагом 1 год с весом 0.5) - _PDHouses_w   
<br>   
   
   
<center>$ \_PDHouses\_w = \ln \left( \dfrac{\_Calc\_PD\_FH\_OM\_Corr_{y-1} \cdot 0.5 + \_Calc\_PD\_FH\_OM\_Corr_y \cdot 0.5}{(1 + \_ProbDef) \dfrac{price1mddy\_alt\_x}{price1mall\_alt\_x}} \right)$</center>


где:   

  - \_Calc\_PD\_FH\_OM\_Corr - Расчет потенциального спроса населения на готовое жилье за счет собственных средств c поправочным коэфф-том, учитывающим насыщение потребности в улучшении жилищных условий (**расчетный**)
  - \_ProbDef - Вероятность дефолта проекта и/или застройщика (**из базы данных**)
  - price1mddy\_alt\_x - Средняя цена 1 кв. м по ДДУ (**из другого блока**)
  - price1mall\_alt\_x - Средняя цена 1 кв. м на рынке готового жилья (**из другого блока**)
  
### Расчет потенциального спроса населения на готовое жилье за счет собственных средств c поправочным коэфф-том, учитывающим насыщение потребности в улучшении жилищных условий - \_Calc\_PD\_FH\_OM\_Corr    
<br>

<center>$\_Calc\_PD\_FH\_OM\_Corr = \_PD+p\_superrich \cdot \_superrich\_buy \cdot Pop\_x \cdot (1-\_p\_self\_improve_{y-1})$</center>

В расчете используются следующие переменные (формулы счета рассчетных переменных приведены ниже):
  - Потенциальный спрос - \_PD (**расчетный**, блок 1.1)
  - Численность населения (среднегодовая) - Pop\_x (**из базы данных**)
  - Доля сверхбогатого населения - p\_superrich (**из базы данных - параметр**)
  - Типовая покупка жилья сверхбогатым населением в год на чел - \_superrich\_buy (**расчетный**, блок 1.2) 
  - Доля семей, фактически улучшивших жилищные условия за скользящие 7 лет за счет собственных средств в отношении к тем, кому доступно улучшение жилищных условий за счет собственных средств, % - \_p\_self\_improve (**расчетный**, блок 1.3)

### 1.1 Расчет  Потенциального спроса \_PD и вспомогательных переменных

Для расчета Потенциального спроса (\_PD) необходимо расчитать следующие переменные:
  - \_comm\_income - Общий объем денежных доходов населения
  - \_part\_income4h - Общего объема денежных доходов, которые могут потратить на жилье (включая ПМ)
  - p\_middleclass - Доля населения с доходами выше порога
  - LivMin - Прожиточный минимум
  - \_val\_m4h - Общий объем денежных доходов, которые могут потратить на жилье (включая ПМ)
  - \_sum\_val\_PM4dem\_h\_om - Суммарный объем ПМ населения, которое может предъявить спрос на жилье за счет собственных средств

а так же переменных, используемых в расчете переменных выше
  - \_p\_income\_superr - Доля доходов сверхбогатого населения
  - \_mid\_level - Порог
  - \_p\_income4h - Доля денежных доходов, которые могут потратить на жилье (включая ПМ) за исключением доходов сверхбогатого населения
  - \_inc\_percapita\_superrich - Среднедушевые доходы сверхбогатого населения

#### 1.1.1 Доля доходов сверхбогатого населения (переменная \_p\_income\_superr)

Для расчета используются следующие входные данные:
  - Среднедушевые денежные доходы населения (в текущих ценах) - *Inc\_x*, загружаются из базы данных
  - Параметр $\mu$ (матожидание) распределения **доходов населения** по уровню среднедушевого среднемесячного денежного дохода в денежном выражении - *MEPop*. ОДновременно это параметр $\mu$ для нормального распределения **населения** по уровню денежных доходов, полученного из логнормального распределения населения по уровню среднедушевых доходов - **данные из ДОМ.РФ**
  - Параметр $\sigma$ (среднеквадратичное отклонение) распределения **доходов населения** по уровню среднедушевого среднемесячного денежного дохода в денежном выражении - *DispPop*. ОДновременно это параметр $\sigma$ для нормального распределения **населения** по уровню денежных доходов, полученного из логнормального распределения населения по уровню среднедушевых доходов - **данные из ДОМ.РФ**
  - Доля сверхбогатого населения - *p_superrich*, загружается из базы данных (наша гипотеза?)
  
**Алгоритм расчета:**
  1. Определяем *Среднедушевые доходы сверхбогатого населения* - *\_inc\_percapita\_superrich*   
  
  $\_inc\_percapita\_superrich=\large{e^{{\mathcal{N}(MEPop,\,DispPop^{2}).ppf(1-p\_superrich)}}} $    
  
  
  где ppf - Percent point function (inverse of cdf — percentiles) **нормального распределения** $\mathcal{N}(MEPop,\,DispPop^{2})   $
  
  2. Полученную величину *_inc_percapita_superrich* используем в качестве X для нахождения доли *доходов сверхбогатого населения (переменная _p_income_superr)* по распределению *доходов населения по уровню среднедушевого среднемесячного денежного дохода в денежном выражении* - [И.Б. Колмаков "Методология расчета и анализа интегральных оценок показателей поляризации денежных доходов населения"](https://voprstat.elpub.ru/jour/article/view/189?locale=ru_RU). 
  pdf (Probability density function) этого распределения имеет вид:
  
   $kolmakov.pdf(x, \sigma, \mu) = \frac{1}{\sigma \sqrt{2\pi}}
                  \exp\left(-\frac{(\log(x)-\mu)^2}{2\sigma^2}\right)$
                  
   искомая величина  
   $ \_p\_income\_superr = 1-\large{\frac{kolmakov.cdf(x, \sigma, \mu)}{Inc\_x}}$
   
   где kolmakov.cdf - сumulative distribution function для распределения выше (интеграл от 0 до x по кривой распределения) с параметрами $\sigma = DispPop$ и $\mu = MEPop$ 

In [24]:
class kolmakov_gen(rv_continuous):
    """ Распределения доходов населения по уровню среднедушевого среднемесячного денежного дохода в денежном выражении """
        
    def _pdf(self, x, sigma, mu):
        Px = np.exp(-(np.log(x)-mu)**2 / (2*sigma**2))
        return Px / (sigma*np.sqrt(2*np.pi))
#         return lognorm.pdf(x, s=sigma, scale=np.exp(mu)) * x
    
    def _cdf(self, x, sigma, mu):
        return quad(self._pdf, 0, x, args=(sigma, mu))[0] 


kolmakov = kolmakov_gen(a=0.0, name='kolmakov', shapes='sigma, mu')

In [25]:
def _calc_superrich_perca_income(x):
    return 1-kolmakov.cdf(x['_inc_percapita_superrich'], sigma=x['DispPop'], mu=x['MEPop'])/x['Inc_x']

pdfWork['_inc_percapita_superrich']=np.exp(norm.ppf(1-pdfWork['p_superrich'], loc=pdfWork['MEPop'], scale=pdfWork['DispPop']))

pdfWork['_p_income_superr']=pdfWork[['_inc_percapita_superrich', 'DispPop', 
                                     'MEPop', 'Inc_x']].apply(_calc_superrich_perca_income, axis=1)

pdfWork.loc[iFirstForecastYear-5:iFirstForecastYear, ['_inc_percapita_superrich', 'DispPop', 'MEPop', 'Inc_x', '_p_income_superr']]

,_inc_percapita_superrich,DispPop,MEPop,Inc_x,_p_income_superr
date,,,,,
2025,178484.503633,0.764492,10.522183,48913.084074,0.083561
2026,192577.432219,0.764492,10.598180,52591.145833,0.080354
2027,207831.418408,0.764492,10.674408,56411.567689,0.074724
2028,223872.019426,0.764492,10.748756,60526.235081,0.071067
2029,240934.615170,0.764492,10.822207,64806.948578,0.066304
2030,258529.121719,0.764492,10.892690,69294.711159,0.063005


#### 1.1.2 Доля населения с доходами выше порога - p_middleclass, Порог - \_mid\_level и Прожиточный минимум - LivMin
<br>   
Расчет доли населения с доходами выше порога делаем аналогично расчету доли сверхбогатого населения 

&emsp;&emsp;&emsp;  $p\_middleclass=1-{\mathcal{N}(MEPop,\,DispPop^{2}).cdf(\_mid\_level)} $    
  
&emsp;  где cdf - umulative distribution function **нормального распределения** $\mathcal{N}(MEPop,\,DispPop^{2})   $

но величину порогового дохода - *_mid_level* (Х для нормального распределения населения по доходам, полученного из логнормального), определяем от:
  - LivMin - Прожиточного минимума (**загружаем из базы данных, расчитываем**)
  - HHAv - Среднего размера  домохозяйства (**из базы данных**)
  - price1mddy\_alt\_x - Средняя цена 1 кв. м по ДДУ (**из другого блока**)
  
  $\begin{equation*}
    LivMin =  
     \begin{cases}
       LivMin_y; \quad \text{при y < прогнозных годов} \\
       LivMin_{y-1} * CPIAv \quad \text{при y>= прогнозных годов }
     \end{cases}
\end{equation*}$

  где CPIAv - ИПЦ, в среднем за год (**из базы данных**)
  
  $\_mid\_level = LivMin + \left(\large{\frac{\frac{\frac{54}{12}}{HHAv}}{7}}\right) * price1mddy\_alt\_x $

In [26]:
idxnan = pdfWork['LivMin'].last_valid_index()
pdfWork.loc[idxnan:, 'LivMin']=pdfWork['LivMin'].combine_first(pdfWork['CPIAv']).loc[idxnan:].cumprod()

pdfWork['_mid_level']=pdfWork['LivMin'] + (54 / 12 / pdfWork['HHAv'] / 7) * pdfWork['price1mddy_alt_x']
pdfWork['p_middleclass']=1-norm.cdf(np.log(pdfWork['_mid_level']), loc=pdfWork['MEPop'], scale=pdfWork['DispPop'])

pdfWork.loc[iFirstForecastYear-5:iFirstForecastYear, ['CPIAv', 'LivMin', 'price1mddy_alt_x', 'HHAv', '_mid_level', 'p_middleclass']]

,CPIAv,LivMin,price1mddy_alt_x,HHAv,_mid_level,p_middleclass
date,,,,,,
2025,1.040099,13888.430924,79864.509286,2.386583,35400.973460,0.524870
2026,1.040317,14448.374728,77294.163838,2.386583,35268.561345,0.566193
2027,1.039478,15018.767882,81884.022783,2.386583,37075.292600,0.579671
2028,1.039971,15619.078396,80003.147175,2.386583,37168.964848,0.616004
2029,1.038610,16222.129239,82889.626557,2.386583,38549.526390,0.634342
2030,1.039028,16855.247745,80764.812206,2.386583,38610.298562,0.667678


#### 1.1.3 Доля денежных доходов, которые могут потратить на жилье (включая ПМ) - \_part\_income4h	

&emsp;&emsp;&emsp; $ \_part\_income4h = 1-\large{\frac{kolmakov.cdf(\_mid\_level)}{Inc\_x}}$
   
   где 
   - kolmakov.cdf - сumulative distribution function для распределения доходов населения по уровню среднедушевого среднемесячного денежного дохода в денежном выражении (см. выше) с параметрами $\mu=MEPop$ и $\sigma = DispPop$
   - \_mid\_level - порог (считаем выше)
   - Inc\_x - Среднедушевые денежные доходы населения (в текущих ценах) (из базы данных)

In [27]:
def _calc_part_income4h(x):
    return 1-kolmakov.cdf(x['_mid_level'], sigma=x['DispPop'], mu=x['MEPop'])/x['Inc_x']

pdfWork['_part_income4h']=pdfWork[['_mid_level', 'DispPop', 'MEPop', 'Inc_x']].apply(_calc_part_income4h, axis=1)

pdfWork.loc[iFirstForecastYear-5:iFirstForecastYear, ['_part_income4h', 'DispPop', 'MEPop', '_mid_level', 'Inc_x']]

,_part_income4h,DispPop,MEPop,_mid_level,Inc_x
date,,,,,
2025,0.792426,0.764492,10.522183,35400.973460,48913.084074
2026,0.820549,0.764492,10.598180,35268.561345,52591.145833
2027,0.828426,0.764492,10.674408,37075.292600,56411.567689
2028,0.850884,0.764492,10.748756,37168.964848,60526.235081
2029,0.861232,0.764492,10.822207,38549.526390,64806.948578
2030,0.879975,0.764492,10.892690,38610.298562,69294.711159


#### 1.1.4 Общий объем денежных доходов населения -\_comm\_income и Доля денежных доходов, которые могут потратить на жилье (включая ПМ) за исключением доходов сверхбогатого населения - \_p\_income4h

$ \_comm\_income = \large \frac{Inc\_x * 12 * Pop\_x}{1000} $ 
  
  
$ \_p\_income4h = \_part\_income4h - \_p\_income\_superr $

In [28]:
pdfWork['_comm_income'] = pdfWork['Inc_x'] * 12 * pdfWork['Pop_x'] / 1e3
pdfWork['_p_income4h'] = pdfWork['_part_income4h'] - pdfWork['_p_income_superr']

pdfWork.loc[iFirstForecastYear-5:iFirstForecastYear, ['_comm_income', '_p_income4h', 'Inc_x', 'Pop_x']]

,_comm_income,_p_income4h,Inc_x,Pop_x
date,,,,
2025,85518.990542,0.708865,48913.084074,145.69890
2026,91745.380125,0.740195,52591.145833,145.37520
2027,98187.876209,0.753702,56411.567689,145.04690
2028,105116.638779,0.779817,60526.235081,144.72600
2029,112314.239573,0.794929,64806.948578,144.42155
2030,119854.017237,0.816971,69294.711159,144.13560


#### 1.1.5  Общий объем денежных доходов, которые могут потратить на жилье (включая ПМ) - \_val\_m4h и Суммарный объем ПМ населения, которое может предъявить спрос на жилье за счет собственных средств - \_sum\_val\_PM4dem\_h\_om
 
$ \_val\_m4h = \_p\_income4h * \_comm\_income$ 

где
  - \_p\_income4h - Доля денежных доходов, которые могут потратить на жилье (включая ПМ) за исключением доходов сверхбогатого населения (**считаем выше**)
  - \_comm\_income - Общий объем денежных доходов населения (**считаем выше**)
  
$ \_sum\_val\_PM4dem\_h\_om = \large \frac{12 * LivMin * p\_middleclass * Pop\_x}{1000}$

где
  - LivMin - Прожиточный минимум (**считаем выше**)
  - p_middleclass - Доля населения с доходами выше порог (**считаем выше**)

In [29]:
pdfWork['_val_m4h'] = pdfWork['_comm_income'] * pdfWork['_p_income4h']
pdfWork['_sum_val_PM4dem_h_om'] = 12 * pdfWork['LivMin'] * pdfWork['p_middleclass'] * pdfWork['Pop_x'] / 1e3


pdfWork.loc[iFirstForecastYear-5:iFirstForecastYear, ['_val_m4h', '_sum_val_PM4dem_h_om', '_comm_income', '_p_income4h', 'LivMin' ,'Pop_x']]

,_val_m4h,_sum_val_PM4dem_h_om,_comm_income,_p_income4h,LivMin,Pop_x
date,,,,,,
2025,60621.435138,12745.066558,85518.990542,0.708865,13888.430924,145.69890
2026,67909.489917,14271.011305,91745.380125,0.740195,14448.374728,145.37520
2027,74004.389815,15153.241507,98187.876209,0.753702,15018.767882,145.04690
2028,81971.708814,16709.618526,105116.638779,0.779817,15619.078396,144.72600
2029,89281.822088,17833.827926,112314.239573,0.794929,16222.129239,144.42155
2030,97917.224564,19465.015265,119854.017237,0.816971,16855.247745,144.13560


#### 1.1.6 Расчет Общего объема денежных доходов, которые могут потратить на жилье - \_val\_incom2house <br>и Потенциального спроса - \_PD

<br>
<center>$\_PD = \dfrac{\_val\_incom2house * 1000}{price1mddy\_alt\_x}$</center>

где 
  - \_val\_incom2house - Общий объем денежных доходов, которые могут потратить на жилье (**считаем выше**)
  - price1mddy\_alt\_x - Средняя цена 1 кв. м по ДДУ (**из другого блока**)
  
  $ \_val\_incom2house = \_val\_m4h - \_sum\_val\_PM4dem\_h\_om $
  
где
  - \_val\_m4h - Общий объем денежных доходов, которые могут потратить на жилье (включая ПМ) (**считаем выше**)
  - \_sum\_val\_PM4dem\_h\_om - Суммарный объем ПМ населения, которое может предъявить спрос на жилье за счет собственных средств (**считаем выше**)

In [30]:
pdfWork['_val_income2house'] = pdfWork['_val_m4h'] - pdfWork['_sum_val_PM4dem_h_om']
pdfWork['_PD'] = pdfWork['_val_income2house'] * 1e3 / pdfWork['price1mddy_alt_x']

pdfWork.loc[iFirstForecastYear-5:iFirstForecastYear, ['_PD', '_val_income2house', '_val_m4h', '_sum_val_PM4dem_h_om']]

,_PD,_val_income2house,_val_m4h,_sum_val_PM4dem_h_om
date,,,,
2025,599.469890,47876.368580,60621.435138,12745.066558
2026,693.952505,53638.478612,67909.489917,14271.011305
2027,718.713447,58851.148308,74004.389815,15153.241507
2028,815.744037,65262.090288,81971.708814,16709.618526
2029,861.965449,71447.994162,89281.822088,17833.827926
2030,971.366207,78452.209299,97917.224564,19465.015265


### 1.2 Типовая покупка жилья сверхбогатым населением в год на чел - \_superrich\_buy
$ \begin{equation*}
\_superrich\_buy =  
 \begin{cases}
   \dfrac{34.71428571}{HHAv_{2008}}; \quad \text{при} \quad 2008<= y <= 2024\\
   \_superrich\_buy_{2024} + \dfrac{\_superrich\_buy_{2030} - \_superrich\_buy_{2024}}{6}\;\;\;;  \quad \text{при} \quad y=2025\\
   2 * \_superrich\_buy_{y-1} - \_superrich\_buy_{y-2};  \quad \text{при} \quad 2025<y<2030\\
   0; \quad \text{при} \quad y=2030
 \end{cases}
\end{equation*} $  

где HHAv - Средний размер домохозяйства (**из базы данных**)

In [31]:
pdfWork['_superrich_buy']=4.5*54/7/pdfWork.loc[iFirstFactYear, 'HHAv']
pdfWork.loc[iFirstForecastYear, '_superrich_buy']=0
pdfWork.loc[2025, '_superrich_buy']= pdfWork.loc[2024, '_superrich_buy'] + (pdfWork.loc[iFirstForecastYear, '_superrich_buy']-pdfWork.loc[2024, '_superrich_buy'])/6

for i in range(2026, 2030):
    pdfWork.loc[i, '_superrich_buy']=2*pdfWork.loc[i-1, '_superrich_buy'] - pdfWork.loc[i-2, '_superrich_buy']

pdfWork.loc[iLastFactYear:iFirstForecastYear, ['_superrich_buy']]

,_superrich_buy
date,
2019,13.276496
2020,13.276496
2021,13.276496
2022,13.276496
2023,13.276496
2024,13.276496
2025,11.063747
2026,8.850997
2027,6.638248


### 1.3 Доля семей, фактически улучшивших жилищные условия за скользящие 7 лет за счет собственных средств в отношении к тем, кому доступно улучшение жилищных условий за счет собственных средств, % - \_p\_self\_improve

$ \_p\_self\_improve = \large\frac{UZUss\_s}{p\_middleclass} $

где
  - UZUss\_s - Доля семей, улучшивших жилищное положение без использования ипотечного кредита нарастающим итогом за 7 лет (**расчет**)
  - p\_middleclass - Доля населения с доходами выше порога (**расчет**)
  
#### 1.2.1 Доля семей, улучшивших жилищное положение без использования ипотечного кредита нарастающим итогом за 7 лет - UZUss\_s

$ UZUss\_s = \large\frac{UZUss\_x}{\_cnt\_hh}$

где
  - UZUss\_x - Число семей, улучшивших жилищное положение без использования ипотечного кредита нарастающим итогом за 7 лет (**из базы данных**))
  - \_cnt\_hh - Число домохозяйств
  
  $ \_cnt\_hh = \large\frac{Pop\_x}{HHAv} $
  
где
  - Pop\_x - Численность населения (среднегодовая) (**из базы данных**))
  - HHAv - Средний размер домохозяйства (**из базы данных**))
  
<div class="alert alert-block alert-warning">
    UZUss_x - Число семей, улучшивших жилищное положение без использования ипотечного кредита нарастающим итогом за 7 лет существуют только для фактических значений. Для прогнозных значений этот показатель расчитывается через модель № 1 "Площадь строящегося жилья, приобретенного за счет собственных средств"<br>
    Т.е. для прогнозных годов этот показатель расчитывается от предидущего года по модели, и используется в качестве регрессора модели для расчета следующего года<br>
    Коэффициенты для модели оцениваются по фактичким значеним - они беруться из базы данных
</div>

In [32]:
pdfWork['_cnt_hh'] = pdfWork['Pop_x']/pdfWork['HHAv']
pdfWork['UZUss_s'] = pdfWork['UZUss_x'] / pdfWork['_cnt_hh']
pdfWork['_p_self_improve'] = pdfWork['UZUss_s'] / pdfWork['p_middleclass']
pdfWork['_p_self_improve']=pdfWork['_p_self_improve'].bfill()
# pdfWork.loc[pdfWork['_p_self_improve'].first_valid_index() -1, '_p_self_improve']=
pdfWork.loc[:iFirstForecastYear, ['_p_self_improve', '_cnt_hh', 'Pop_x', 'HHAv', 'UZUss_x', 'UZUss_s', 'p_middleclass']]

,_p_self_improve,_cnt_hh,Pop_x,HHAv,UZUss_x,UZUss_s,p_middleclass
date,,,,,,,
2004,0.469431,53.815618,144.067300,2.677054,NaN,NaN,NaN
2005,0.469431,53.924648,143.518814,2.661470,NaN,NaN,0.391122
2006,0.469431,54.064936,143.049637,2.645886,NaN,NaN,0.383405
2007,0.469431,54.292297,142.805114,2.630302,NaN,NaN,0.374674
2008,0.469431,54.591889,142.742366,2.614717,9.763312,0.178842,0.380976
2009,0.437808,54.935752,142.785348,2.599133,10.421675,0.189707,0.433310
2010,0.420668,55.291945,142.849468,2.583549,11.124432,0.201194,0.478274
2011,0.436982,56.191877,142.960908,2.544156,11.874579,0.211322,0.483595
2012,0.447104,57.171768,143.201721,2.504763,12.675309,0.221706,0.495871


### 1.4 Расчет потенциального спроса населения на готовое жилье за счет собственных средств c поправочным коэфф-том, учитывающим насыщение потребности в улучшении жилищных условий - \_Calc\_PD\_FH\_OM\_Corr  и Потенциального спроса на житье (без лага с весом 0.5, с лагом 1 год с весом 0.5) - _PDHouses_w   (регрессор модели)
<br>

<center>$\_Calc\_PD\_FH\_OM\_Corr = \_PD+p\_superrich \cdot \_superrich\_buy \cdot Pop\_x \cdot (1-\_p\_self\_improve_{y-1})$</center>
<br>   
   
   
<center>$ \_PDHouses\_w = \ln \left( \dfrac{\_Calc\_PD\_FH\_OM\_Corr_{y-1} \cdot 0.5 + \_Calc\_PD\_FH\_OM\_Corr_y \cdot 0.5}{(1 + p\_ProbDef) \dfrac{price1mddy\_alt\_x}{price1mall\_alt\_x}} \right)$</center>

### 1.5 ...и добавляем дамми

In [33]:
pdfWork=pdfWork.loc[iFirstFactYear-1:, :]

pdfWork['_Calc_PD_FH_OM_Corr'] = (pdfWork['_PD'] + pdfWork['p_superrich'] * pdfWork['_superrich_buy'] 
                                  * pdfWork['Pop_x']) * (1 - pdfWork['_p_self_improve'].shift(1))

pdfWork['_PDHouses_w']=(np.log( 
    (pdfWork['_Calc_PD_FH_OM_Corr'].shift(1)*0.5 + pdfWork['_Calc_PD_FH_OM_Corr']*0.5) 
    / ((1+pdfWork['p_ProbDef']) * pdfWork['price1mddy_alt_x'] / pdfWork['price1mall_alt_x']) )
    )

pdfWork['D']=0
pdfWork.loc[ (2014, 2019), 'D']=1
pdfWork[['_PDHouses_w', '_Calc_PD_FH_OM_Corr', '_PD', 'p_superrich', '_superrich_buy', 'Pop_x', '_p_self_improve', 'p_ProbDef', 'D']]

,_PDHouses_w,_Calc_PD_FH_OM_Corr,_PD,p_superrich,_superrich_buy,Pop_x,_p_self_improve,p_ProbDef,D
date,,,,,,,,,
2007,NaN,NaN,303.632341,0.02,13.276496,142.805114,0.469431,0.10,0
2008,NaN,186.476606,313.563054,0.02,13.276496,142.742366,0.469431,0.10,0
2009,5.395987,229.236901,394.144941,0.02,13.276496,142.785348,0.437808,0.10,0
2010,5.615056,287.937369,474.238226,0.02,13.276496,142.849468,0.420668,0.10,0
2011,5.767168,306.700655,491.443309,0.02,13.276496,142.960908,0.436982,0.10,0
2012,5.827185,310.723911,513.865245,0.02,13.276496,143.201721,0.447104,0.10,0
2013,5.855479,320.774621,542.065980,0.02,13.276496,143.506995,0.448079,0.10,0
2014,5.881504,325.332256,550.663152,0.02,13.276496,146.090613,0.461305,0.10,1
2015,5.881621,332.528273,578.410087,0.02,13.276496,146.405999,0.459945,0.10,0


## 2 Оценка модели  1 - Площадь строящегося жилья, приобретенного за счет собственных средств
<br>

<center>  $ squareddy\_ss\_x \sim \_PDHouses\_w + Ln(Unmpl\_s) + D_{(2014, 2019)} - 1 $  </center>


In [46]:
pdfXY = pdfWork.loc[iFirstFactYear:iLastFactYear, ['_PDH', '_PDH_l', 'squareddy_ss_x', 'Unmpl_s', 'D', '_PDHouses_w']].dropna()

resSSSX = smf.ols(formula='squareddy_ss_x ~ _PDH + _PDH_l + np.log(Unmpl_s) + D - 1', missing='drop', data=pdfXY).fit()
print(resSSSX.summary())

                                 OLS Regression Results                                
Dep. Variable:         squareddy_ss_x   R-squared (uncentered):                   0.986
Model:                            OLS   Adj. R-squared (uncentered):              0.977
Method:                 Least Squares   F-statistic:                              108.3
Date:                Wed, 03 Feb 2021   Prob (F-statistic):                    1.01e-05
Time:                        21:40:43   Log-Likelihood:                         -17.824
No. Observations:                  10   AIC:                                      43.65
Df Residuals:                       6   BIC:                                      44.86
Df Model:                           4                                                  
Covariance Type:            nonrobust                                                  
                      coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------

In [35]:
0,60
-0,66
0,39
pdfXY

,_PDHouses_w,squareddy_ss_x,Unmpl_s,D
date,,,,
2010,5.615056,7.060626,7.344543,0
2011,5.767168,9.382939,6.495643,0
2012,5.827185,11.554604,5.458664,0
2013,5.855479,8.939837,5.478040,0
2014,5.881504,18.388684,5.156324,1
2015,5.881621,13.821867,5.567361,0
2016,5.890488,9.995342,5.537444,0
2017,5.892738,10.450514,5.203688,0
2018,5.948093,12.838971,4.801915,0


In [43]:
pdfWork['_PDH']=pdfWork['_PDHouses_w']*0.5
pdfWork['_PDH_l']=pdfWork['_PDHouses_w'].shift(1)*0.5

pdfWork[['_PDHouses_w', 'squareddy_ss_x', 'Unmpl_s', 'D', '_PDH', '_PDH_l']]

,_PDHouses_w,squareddy_ss_x,Unmpl_s,D,_PDH,_PDH_l
date,,,,,,
2007,NaN,NaN,6.029393,0,NaN,NaN
2008,NaN,NaN,6.241192,0,NaN,NaN
2009,5.395987,NaN,8.252236,0,2.697994,NaN
2010,5.615056,7.060626,7.344543,0,2.807528,2.697994
2011,5.767168,9.382939,6.495643,0,2.883584,2.807528
2012,5.827185,11.554604,5.458664,0,2.913592,2.883584
2013,5.855479,8.939837,5.478040,0,2.927740,2.913592
2014,5.881504,18.388684,5.156324,1,2.940752,2.927740
2015,5.881621,13.821867,5.567361,0,2.940810,2.940752
